In [1]:
import datetime
import logging
import os
import psycopg2
from subprocess import call

import numpy as np
import pandas as pd

In [2]:
Employee = pd.read_csv('Employee.csv')

In [3]:
Industry = pd.read_csv('Industry.csv')

## Запрос 1

Вывести список жанров книг
эта инфармация поможет нам ариентироваться в данных которые мы будем получать в дальнейших запроса


#### SQL release
SELECT * FROM library

#### Pandas release

In [4]:
library_list = pd.read_csv('library.csv')
library_list['genre'].unique()

array(['science fiction', 'non fiction', 'historical novel', 'adventures',
       'romantic', 'horror', 'drama ', 'documental ', 'detective'],
      dtype=object)

## Запрос 2

Выполняем первый аналитический запрос:
Вывести три самых популярных жанра литературы (genre) среди выборки из 251 читателя (разного возраста, пола, образование, сферы деятельности) данная выборка реперезентативна она, в ней представлен широкий круг респондентов
Полученные данные помогут понять какие жанры литературы сейчас самые читаемые, данная нформация в дальнейшем поможет фрагментировать рынок при построениии бизнесс моделей издательств

#### SQL release

WITH tmp_tab as (SELECT * FROM library JOIN library_card ON library.book_id=library_card.book_genre_id) SELECT genre, COUNT(book_genre_id) as top_genre FROM tmp_tab GROUP BY genre ORDER BY top_genre DESC LIMIT 3;

#### Python release



In [5]:
# используем файлы library.csv и library_card.csv

library = pd.read_csv('library.csv')
library_card = pd.read_csv('library_card.csv')

делаем join двух таблиц library и library_card с помощью groupby() группируем данные и считаем сколько книг приходится на каждый жанр книг с помощью оператора count()

In [7]:
df_2 = library.merge(library_card, left_on='book_id', right_on='book_genre_id', how='outer')\
                                                                                        .groupby('genre').count()
df_2[['book_genre_id']].sort_values(by = ['book_genre_id'],  ascending = False )[:3]

,book_genre_id
genre,
romantic,39
adventures,36
non fiction,31


## Запрос 3
Выполняем второй аналитический запрос: Вывести топ-3 жанра которые читают женщины старше 50 лет
Полученные данные помогут понять какие жанры литературы популярны у женьщин старшего возраста

#### SQL release
WITH tmp_tab as 
            (SELECT * FROM library JOIN books_after_50 ON library.book_id=books_after_50.book_genre_id) 
            SELECT DISTINCT genre FROM tmp_tab WHERE age>50 and gender LIKE '%Female' LIMIT 3;

#### Python release

In [8]:
#используем данные из таблиц

library = pd.read_csv('library.csv')
Employee = pd.read_csv('Employee.csv')

In [13]:
books_after_50 = library_card.merge(Employee, left_on='nameid', right_on='name_id', how='outer')


df_4 = library.merge(books_after_50, left_on='book_id', \
                     right_on='book_genre_id', how='outer').sort_values(by = ['book_genre_id'])

fd = df_4[(df_4['age']>=50) & (df_4['gender'] == 'Female')]\
                                .groupby('genre').sum().sort_values(by = ['book_genre_id'], ascending = False )
    
    
fd['book_genre_id'][:3]

genre
romantic      30
adventures    20
horror        18
Name: book_genre_id, dtype: int64

## Запрос 4
Определяем минимальный возраст респондента (среди респондентов нашего опроса из запроса 1)



#### SQL release

SELECT Min(age) as min_age FROM employee;

#### Python release

для поиска минимального возраста используем оператор min()

In [10]:
# используем таблицу employee
stuff_age = pd.read_csv('Employee.csv')
min(stuff_age['age']) 

17

## Запрос 5
Выполняем третий аналитический запрос: Вывести средний значения продолжительности чтения книг в зависимости от жанра
Эти данные помогут издателям понять сколько времени читатели разных жанров тратят времени на чтение книг, данную информацию можно использовать при принятии решения какое количиство страниц оптимально для книг представленных в каждом жанре

In [18]:
df_2 = library.merge(library_card, left_on='book_id', right_on='book_genre_id', how='outer')\
                                                                                        .groupby('genre').mean()
df_2[['reading_time_days']].sort_values(by = ['reading_time_days'],  ascending = False )

,reading_time_days
genre,
historical novel,28.680000
drama,28.500000
adventures,27.166667
romantic,25.769231
horror,22.520000
science fiction,21.521739
detective,21.115385
non fiction,17.838710
documental,16.500000
